# Import Libraries

In [1]:
import healpy as hp
import numpy as np
from pathlib import Path
from matplotlib import pyplot as plt

In [6]:
from src.info_extractor import InfoExtractor
from src.patch_analyser import PatchAnalyser
from src.fullsky_analyser import FullSkyAnalyser
from analysis import KappaAnalyser
from src.patch_processor import PatchProcessor
from src.utils import load_config, filter_config, setup_logging

2024-10-09 08:22:54,317 - INFO - Note: NumExpr detected 52 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


# Setup the environment

In [3]:
workdir = Path("/lustre/work/akira.tokiwa/Projects/LensingSSC")
config_file = workdir / "configs" / "config_default.yaml"
sample_dir = workdir / "data" / "tiled" / "proc2048_nc1024_size625_nsteps60_s401_10tiled"

In [4]:
config = load_config(config_file)
setup_logging()

In [8]:
filtered_config_pp = filter_config(config, PatchProcessor)
pp = PatchProcessor(**filtered_config_pp)

filtered_config_pa = filter_config(config, PatchAnalyser)
pa = PatchAnalyser(pp, **filtered_config_pa)

filtered_config_fsa = filter_config(config, FullSkyAnalyser)
fsa = FullSkyAnalyser(**filtered_config_fsa)

ka = KappaAnalyser(sample_dir, pa, fsa)

2024-10-09 08:23:12,553 - INFO - Initialized PatchProcessor with npatch=273, patch_size_deg=10.0, xsize=2048
2024-10-09 08:23:12,554 - INFO - Initializing FlatPatchAnalyser with nbin=15, lmin=300, lmax=3000
2024-10-09 08:23:12,564 - INFO - FullSkyAnalyser initialized with nside=8192, nbin=15, lmin=300, lmax=3000
2024-10-09 08:23:12,565 - INFO - Output directories created


In [9]:
# ka.analyse()

# Sample run

In [11]:
patch_kappa_path = ka.patch_kappa_paths[0]
tmp_kappa_patches = np.load(patch_kappa_path)
tmp_snr_paths = ka._filter_paths_by_input(ka.patch_snr_paths, patch_kappa_path)
patch_snr_path = tmp_snr_paths[0]
fname = Path(patch_snr_path).name.replace('patches', 'analysis')
snr_patches = np.load(patch_snr_path)
data = ka.pa.process_patches(tmp_kappa_patches, snr_patches)

In [23]:
# sk0, sk1, sk2, kur0, kur1, kur2, kur3 are scalars
# all the other values are the same length: pa.nbin
# The order of the values is equilateral, isosceles, squeezed, clkk,
#            sk0, sk1, sk2, kur0, kur1, kur2, kur3,
#            pdf_vals, peaks, minima, v0, v1, v2

# separate the data
equilateral = data[:, pa.nbin * 0:pa.nbin * 1]
isosceles = data[:, pa.nbin * 1:pa.nbin * 2]
squeezed = data[:, pa.nbin * 2:pa.nbin * 3]
clkk = data[:, pa.nbin * 3:pa.nbin * 4]
sk0 = data[:, pa.nbin * 4]
sk1 = data[:, pa.nbin * 4 + 1]
sk2 = data[:, pa.nbin * 4 + 2]
kur0 = data[:, pa.nbin * 4 + 3]
kur1 = data[:, pa.nbin * 4 + 4]
kur2 = data[:, pa.nbin * 4 + 5]
kur3 = data[:, pa.nbin * 4 + 6]
pdf_vals = data[:, pa.nbin * 4 + 7:pa.nbin * 5 + 7]
peaks = data[:, pa.nbin * 5 + 7:pa.nbin * 6 + 7]
minima = data[:, pa.nbin * 6 + 7:pa.nbin * 7 + 7]
v0 = data[:, pa.nbin * 7 + 7:pa.nbin * 8 + 7]
v1 = data[:, pa.nbin * 8 + 7:pa.nbin * 9 + 7]
v2 = data[:, pa.nbin * 9 + 7:pa.nbin * 10 + 7]

In [14]:
global_std = np.std(snr_patches)

In [15]:
from lenstools import ConvergenceMap
from astropy import units as u

patch_pixels = tmp_kappa_patches[0]
patch_snr_pixels = snr_patches[0]

# Process kappa (convergence) map
conv_map = ConvergenceMap(patch_pixels, angle=pa.patch_size * u.deg)
equilateral, isosceles, squeezed = pa._compute_bispectrum(conv_map)
clkk = pa._compute_power_spectrum(conv_map)
sk0, sk1, sk2, kur0, kur1, kur2, kur3 = pa._compute_moments(conv_map, global_std)

# Process SNR map
snr_map = ConvergenceMap(patch_snr_pixels/global_std, angle=pa.patch_size * u.deg)
pdf_vals = pa._compute_pdf(snr_map)
peaks = pa._compute_peak_statistics(snr_map, is_minima=False)
minima = pa._compute_peak_statistics(snr_map, is_minima=True)
v0,v1,v2 = pa._compute_minkowski_functionals(snr_map)

# Check the output files